In [63]:
%pylab inline
%matplotlib inline

# Global Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import os
import sys
import pickle
from pprint import pprint
from time import time
import datetime
from time import gmtime, strftime
import statsmodels.api as sm
from patsy import dmatrices

# Scikit-Learn imports
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression


from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split

from sklearn.cross_validation import cross_val_score
from sklearn import metrics

# Local Imports
path = str(os.path.expanduser('~')) + '/git/predictEPL/config'
sys.path.append(path)
import paths

sys.path.append(paths.UTILS)
import useful_methods

Populating the interactive namespace from numpy and matplotlib


In [3]:
date_now = strftime("%Y-%m-%d %H:%M:%S", gmtime()).replace(" ", "_")

# ***************************************************
# [Step 1]: Data Load
# ***************************************************

# Read Data
df = useful_methods.csv_dic_df(paths.DATA_HOME + "EPL/all_game_emolex_counted.csv")

# Manipulations

dic_res ={
    'home_win': 1,
    'away_win': 0,
    'draw': 2
}

def Labeling(goal_diff):
    if goal_diff > 0:
        return 1
    elif goal_diff < 0:
        return 0
    else:
        return 2

df.score_ft_home = [int(score_ft_home) for score_ft_home in df.score_ft_home]
df.score_ft_away = [int(score_ft_away) for score_ft_away in df.score_ft_away]

df.pn_home = [np.array([float(pn) for pn in pn_home[1:-1].split(',')]) for pn_home in list(df.pn_home)]
df.pn_away = [np.array([float(pn) for pn in pn_away[1:-1].split(',')]) for pn_away in list(df.pn_away)]

df.emolex_home = [np.array([float(emo) for emo in emolex_home[1:-1].split(',')]) for emolex_home in list(df.emolex_home)]
df.emolex_away = [np.array([float(emo) for emo in emolex_away[1:-1].split(',')]) for emolex_away in list(df.emolex_away)]

df['goal_diff'] = df.score_ft_home - df.score_ft_away
df['result'] = [Labeling(goal_diff) for goal_diff in df.goal_diff]
# df = df[df.result != 2].copy().reset_index(drop=True)
df

,GW,away_team,date,emolex_away,emolex_home,home_team,pn_away,pn_home,score_ft_away,score_ft_home,score_ht_away,score_ht_home,time,goal_diff,result
0,5,Chelsea,Sat. 12 Sep.,"[1732.0, 1670.0, 1436.0, 1569.0, 1079.0, 2217....","[405.0, 287.0, 192.0, 322.0, 291.0, 484.0, 530...",Everton,"[3901.0, 3699.0]","[1009.0, 647.0]",1,3,1,2,11:45,2,1
1,5,Bournemouth,Sat. 12 Sep.,"[36.0, 36.0, 25.0, 22.0, 27.0, 50.0, 37.0, 37.0]","[67.0, 64.0, 22.0, 54.0, 122.0, 194.0, 122.0, ...",Norwich,"[112.0, 53.0]","[331.0, 100.0]",1,3,0,1,14:00,2,1
2,5,Swansea,Sat. 12 Sep.,"[24.0, 23.0, 10.0, 20.0, 22.0, 51.0, 33.0, 46.0]","[38.0, 22.0, 18.0, 20.0, 57.0, 66.0, 52.0, 58.0]",Watford,"[78.0, 36.0]","[98.0, 55.0]",0,1,0,0,14:00,1,1
3,5,Southampton,Sat. 12 Sep.,"[23.0, 28.0, 17.0, 23.0, 75.0, 65.0, 68.0, 67.0]","[56.0, 30.0, 38.0, 32.0, 43.0, 80.0, 48.0, 94.0]",WestBromwich,"[154.0, 62.0]","[116.0, 116.0]",0,0,0,0,14:00,0,2
4,5,Stoke,Sat. 12 Sep.,"[82.0, 59.0, 40.0, 47.0, 75.0, 130.0, 107.0, 8...","[603.0, 445.0, 421.0, 432.0, 1039.0, 1681.0, 1...",Arsenal,"[255.0, 113.0]","[2529.0, 938.0]",0,2,0,1,14:00,2,1
5,5,City,Sat. 12 Sep.,"[1029.0, 1358.0, 538.0, 805.0, 441.0, 378.0, 3...","[146.0, 113.0, 60.0, 80.0, 100.0, 190.0, 145.0...",Crystal,"[975.0, 1708.0]","[299.0, 225.0]",1,0,0,0,14:00,-1,0
6,5,Liverpool,Sat. 12 Sep.,"[1743.0, 1955.0, 1644.0, 1838.0, 1207.0, 2875....","[5228.0, 3026.0, 2403.0, 2447.0, 2705.0, 7167....",United,"[4881.0, 4464.0]","[11807.0, 9999.0]",1,3,0,0,16:30,2,1
7,5,Tottenham,Sun. 13 Sep.,"[379.0, 365.0, 248.0, 315.0, 297.0, 511.0, 421...","[158.0, 152.0, 87.0, 120.0, 205.0, 329.0, 218....",Sunderland,"[1117.0, 815.0]","[498.0, 330.0]",1,0,0,0,12:30,-1,0
8,5,Villa,Sun. 13 Sep.,"[195.0, 233.0, 203.0, 191.0, 384.0, 725.0, 881...","[70.0, 75.0, 74.0, 65.0, 99.0, 122.0, 99.0, 10...",Leicester,"[1550.0, 480.0]","[276.0, 163.0]",2,3,1,0,15:00,1,1
9,5,Newcastle,Mon. 14 Sep.,"[589.0, 697.0, 473.0, 643.0, 418.0, 919.0, 574...","[96.0, 92.0, 64.0, 68.0, 147.0, 401.0, 297.0, ...",WestHam,"[1560.0, 1563.0]","[655.0, 202.0]",0,2,0,1,19:00,2,1


In [41]:
df = df[df.result != 2].copy().reset_index(drop=True)
dta = pd.DataFrame()

dta['pos_home'] = [pn_home[0] / sum(pn_home)  for pn_home in df.pn_home]
dta['neg_home'] = [pn_home[1] / sum(pn_home)  for pn_home in df.pn_home]

dta['pos_away'] = [pn_away[0] / sum(pn_away)  for pn_away in df.pn_away]
dta['neg_away'] = [pn_away[1] / sum(pn_away)  for pn_away in df.pn_away]


dta['anger_home'] = [emolex[0] / sum(emolex) for emolex in df.emolex_home]
dta['fear_home'] = [emolex[1] / sum(emolex) for emolex in df.emolex_home]
dta['disgust_home'] = [emolex[2] / sum(emolex) for emolex in df.emolex_home]
dta['sadness_home'] = [emolex[3] / sum(emolex)  for emolex in df.emolex_home]
dta['surprise_home'] = [emolex[4] / sum(emolex)  for emolex in df.emolex_home]
dta['trust_home'] = [emolex[5] / sum(emolex)  for emolex in df.emolex_home]
dta['joy_home'] = [emolex[6] / sum(emolex)  for emolex in df.emolex_home]
dta['anticipation_home'] = [emolex[7] / sum(emolex)  for emolex in df.emolex_home]


dta['anger_away'] = [emolex[0] / sum(emolex)  for emolex in df.emolex_away]
dta['fear_away'] = [emolex[1] / sum(emolex)  for emolex in df.emolex_away]
dta['disgust_away'] = [emolex[2] / sum(emolex)  for emolex in df.emolex_away]
dta['sadness_away'] = [emolex[3] / sum(emolex)  for emolex in df.emolex_away]
dta['surprise_away'] = [emolex[4] / sum(emolex)  for emolex in df.emolex_away]
dta['trust_away'] = [emolex[5] / sum(emolex)  for emolex in df.emolex_away]
dta['joy_away'] = [emolex[6] / sum(emolex)  for emolex in df.emolex_away]
dta['anticipation_away'] = [emolex[7] / sum(emolex)  for emolex in df.emolex_away]


dta['score_ht_home'] = [int(score_ht_home) for score_ht_home in df.score_ht_home]
dta['score_ht_away'] = [int(score_ht_away) for score_ht_away in df.score_ht_away]


# 'home_win': 1, 'away_win': 0, 'draw': 2
dta['result'] = df.result
dta

,pos_home,neg_home,pos_away,neg_away,anger_home,fear_home,disgust_home,sadness_home,surprise_home,trust_home,...,fear_away,disgust_away,sadness_away,surprise_away,trust_away,joy_away,anticipation_away,score_ht_home,score_ht_away,result
0,0.609300,0.390700,0.513289,0.486711,0.136824,0.096959,0.064865,0.108784,0.098311,0.163514,...,0.125971,0.108320,0.118353,0.081391,0.167232,0.124085,0.143999,2,1,1
1,0.767981,0.232019,0.678788,0.321212,0.082209,0.078528,0.026994,0.066258,0.149693,0.238037,...,0.133333,0.092593,0.081481,0.100000,0.185185,0.137037,0.137037,1,0,1
2,0.640523,0.359477,0.684211,0.315789,0.114804,0.066465,0.054381,0.060423,0.172205,0.199396,...,0.100437,0.043668,0.087336,0.096070,0.222707,0.144105,0.200873,0,0,1
3,0.729449,0.270551,0.692935,0.307065,0.085133,0.062826,0.059438,0.060991,0.146689,0.237329,...,0.093800,0.063593,0.074722,0.119237,0.206677,0.170111,0.141494,1,0,1
4,0.570611,0.429389,0.363399,0.636601,0.145563,0.112662,0.059821,0.079761,0.099701,0.189432,...,0.228620,0.090572,0.135522,0.074242,0.063636,0.057239,0.176936,0,0,0
5,0.541456,0.458544,0.522311,0.477689,0.150100,0.086879,0.068992,0.070256,0.077663,0.205771,...,0.123414,0.103781,0.116028,0.076195,0.181491,0.136039,0.153021,0,0,1
6,0.601449,0.398551,0.578157,0.421843,0.104153,0.100198,0.057350,0.079103,0.135135,0.216875,...,0.123771,0.084096,0.106816,0.100712,0.173279,0.142760,0.140047,0,0,0
7,0.628702,0.371298,0.763547,0.236453,0.099010,0.106082,0.104668,0.091938,0.140028,0.172560,...,0.065193,0.056799,0.053442,0.107443,0.202854,0.246503,0.213206,0,1,1
8,0.764294,0.235706,0.499520,0.500480,0.066992,0.064201,0.044662,0.047453,0.102582,0.279833,...,0.136586,0.092691,0.126004,0.081913,0.180090,0.112483,0.154811,1,0,1
9,0.563569,0.436431,0.410524,0.589476,0.117257,0.098728,0.126383,0.092459,0.085177,0.157356,...,0.120058,0.174138,0.128831,0.077995,0.142050,0.088811,0.091936,0,0,1


In [34]:
# 'home_win': 1, 'away_win': 0, 'draw': 2
dta.groupby('result').mean()

,pos_home,neg_home,pos_away,neg_away,anger_home,fear_home,disgust_home,sadness_home,surprise_home,trust_home,...,anger_away,fear_away,disgust_away,sadness_away,surprise_away,trust_away,joy_away,anticipation_away,score_ht_home,score_ht_away
result,,,,,,,,,,,,,,,,,,,,,
0,0.594051,0.405949,0.626635,0.373365,0.121734,0.110601,0.074734,0.095972,0.111555,0.183388,...,0.114128,0.102344,0.067767,0.100756,0.102807,0.195737,0.153609,0.162851,0.375000,0.833333
1,0.656051,0.343949,0.557368,0.442632,0.104693,0.091810,0.058235,0.084646,0.110660,0.209230,...,0.125709,0.115874,0.085068,0.112246,0.100829,0.171509,0.135504,0.153261,1.121622,0.202703


## Prepare Data for Logistic Regression

In [57]:
y, X = dmatrices('result ~ \
    pos_home + neg_home + pos_away + neg_away + \
    score_ht_home + score_ht_away + \
    anger_home + fear_home + disgust_home + sadness_home + \
    surprise_home + trust_home + joy_home + anticipation_home + \
    anger_away + fear_away + disgust_away + sadness_away + \
    surprise_away + trust_away + joy_away + anticipation_away',
    dta, return_type="dataframe")

# flatten y into a 1-D array
y = np.ravel(y)

print(X.columns)
X.head()

Index(['Intercept', 'pos_home', 'neg_home', 'pos_away', 'neg_away',
       'score_ht_home', 'score_ht_away', 'anger_home', 'fear_home',
       'disgust_home', 'sadness_home', 'surprise_home', 'trust_home',
       'joy_home', 'anticipation_home', 'anger_away', 'fear_away',
       'disgust_away', 'sadness_away', 'surprise_away', 'trust_away',
       'joy_away', 'anticipation_away'],
      dtype='object')


,Intercept,pos_home,neg_home,pos_away,neg_away,score_ht_home,score_ht_away,anger_home,fear_home,disgust_home,...,joy_home,anticipation_home,anger_away,fear_away,disgust_away,sadness_away,surprise_away,trust_away,joy_away,anticipation_away
0,1,0.609300,0.390700,0.513289,0.486711,2,1,0.136824,0.096959,0.064865,...,0.179054,0.151689,0.130648,0.125971,0.108320,0.118353,0.081391,0.167232,0.124085,0.143999
1,1,0.767981,0.232019,0.678788,0.321212,1,0,0.082209,0.078528,0.026994,...,0.149693,0.208589,0.133333,0.133333,0.092593,0.081481,0.100000,0.185185,0.137037,0.137037
2,1,0.640523,0.359477,0.684211,0.315789,0,0,0.114804,0.066465,0.054381,...,0.157100,0.175227,0.104803,0.100437,0.043668,0.087336,0.096070,0.222707,0.144105,0.200873
3,1,0.729449,0.270551,0.692935,0.307065,1,0,0.085133,0.062826,0.059438,...,0.172102,0.175491,0.130366,0.093800,0.063593,0.074722,0.119237,0.206677,0.170111,0.141494
4,1,0.570611,0.429389,0.363399,0.636601,0,0,0.145563,0.112662,0.059821,...,0.144566,0.168495,0.173232,0.228620,0.090572,0.135522,0.074242,0.063636,0.057239,0.176936


In [59]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegressionCV(multi_class='multinomial')
model = model.fit(X, y)

# check the accuracy on the training set
print(model.score(X, y))


# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegressionCV(multi_class='multinomial'), X, y, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())

0.801652892562
[ 0.76923077  0.76923077  0.76923077  0.76923077  0.83333333  0.83333333
  0.75        0.72727273  0.72727273  0.63636364]
0.75844988345


In [52]:
model.coef_

array([[ 0.05937872,  0.12889578, -0.06951706,  0.03317753,  0.02620119,
         1.43530737, -1.7442204 ,  0.00246387, -0.00648216, -0.05280062,
         0.14011187, -0.11466844,  0.0537984 , -0.0239923 ,  0.06094811,
        -0.22511524, -0.10057716, -0.04388674, -0.20775048,  0.26169242,
        -0.0029011 ,  0.17436063,  0.20355637]])

In [51]:
coef = pd.DataFrame()
coef['var'] = X.columns
coef['score'] = np.transpose(model.coef_)
coef

,var,score
0,Intercept,0.059379
1,pos_home,0.128896
2,neg_home,-0.069517
3,pos_away,0.033178
4,neg_away,0.026201
5,score_ht_home,1.435307
6,score_ht_away,-1.744220
7,anger_home,0.002464
8,fear_home,-0.006482
9,disgust_home,-0.052801


## Model Evaluation Using a Validation Set

In [26]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [27]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print(predicted)

[ 1.  1.  1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  0.  1.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.
  1.]


In [28]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print(probs)

[[ 0.42838278  0.57161722]
 [ 0.35605579  0.64394421]
 [ 0.45729187  0.54270813]
 [ 0.33739516  0.66260484]
 [ 0.52311483  0.47688517]
 [ 0.38505406  0.61494594]
 [ 0.511596    0.488404  ]
 [ 0.29387505  0.70612495]
 [ 0.35660616  0.64339384]
 [ 0.31029047  0.68970953]
 [ 0.36402715  0.63597285]
 [ 0.41242044  0.58757956]
 [ 0.46405293  0.53594707]
 [ 0.47402904  0.52597096]
 [ 0.35738598  0.64261402]
 [ 0.41391875  0.58608125]
 [ 0.35829015  0.64170985]
 [ 0.43987245  0.56012755]
 [ 0.3798032   0.6201968 ]
 [ 0.59890463  0.40109537]
 [ 0.34640377  0.65359623]
 [ 0.41623275  0.58376725]
 [ 0.45496113  0.54503887]
 [ 0.53926925  0.46073075]
 [ 0.35894869  0.64105131]
 [ 0.53070609  0.46929391]
 [ 0.31515686  0.68484314]
 [ 0.48426414  0.51573586]
 [ 0.51347575  0.48652425]
 [ 0.42579091  0.57420909]
 [ 0.31970059  0.68029941]
 [ 0.45299943  0.54700057]
 [ 0.43038888  0.56961112]
 [ 0.42089156  0.57910844]
 [ 0.43015197  0.56984803]
 [ 0.38263135  0.61736865]
 [ 0.42777012  0.57222988]]


In [29]:
# generate evaluation metrics
print(metrics.accuracy_score(y_test, predicted))
print(metrics.roc_auc_score(y_test, probs[:, 1]))

0.756756756757
0.772727272727


In [30]:
print(metrics.confusion_matrix(y_test, predicted))
print(metrics.classification_report(y_test, predicted))

[[ 4  7]
 [ 2 24]]
             precision    recall  f1-score   support

        0.0       0.67      0.36      0.47        11
        1.0       0.77      0.92      0.84        26

avg / total       0.74      0.76      0.73        37



## Model Evaluation Using Cross-Validation

In [ ]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())